In [9]:
import logging
import numpy as np
import pandas as pd
import ujson as json
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import utils
import torchvision.transforms as transforms
import torchvision

from skimage import io
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from transformers import DistilBertTokenizer, DistilBertConfig
from args import get_setup_args
from collections import Counter
from models import DistilBERT

news = 'liberty'
image_path = './data/pictures/liberty/0.jpg'
data_path = './data/train_raw.npz'
train_record_file = './data/train.npz'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
transform = transforms.Compose([transforms.Resize((224, 224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])])
train_dataset = utils.NEWS(train_record_file, transform)

In [11]:
config = DistilBertConfig()
DistilBERT = DistilBERT(config, 8)

In [5]:
input_idxs, atten_masks, images, ids, y = train_dataset[0]

In [23]:
images.dtype

torch.float32

In [22]:
DistilBERT(input_idxs.unsqueeze(0), atten_masks.unsqueeze(0), images.unsqueeze(0))

torch.Size([1, 768])
torch.Size([1, 512])


tensor([[-2.7932, -2.1477, -1.1825, -2.6472, -2.4500, -2.0023, -2.1204, -2.2696]],
       grad_fn=<LogSoftmaxBackward>)